In [97]:
import sys
import os
import grid2op
from lightsim2grid import LightSimBackend
import inspect
import pdb
import numpy as np
import pandas as pd
from grid2op.Reward import GameplayReward

# from grid2op.gym_compat import GymEnv
# from l2rpn_baselines.utils import GymEnvWithRecoWithDN, GymEnvWithHeuristics
# from grid2op.Chronics.multiFolder import Multifolder
# from GymEnvStorage import GymEnvStorage
from GymEnvWithSetPoint import GymEnvWithSetPoint
from grid2op.gym_compat import BoxGymActSpace, BoxGymObsSpace

# from grid2op.Observation import BaseObservation
# from typing import Tuple, Dict

# from l2rpn_baselines.utils import GymAgent
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import json

/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
env = grid2op.make("educ_case14_storage_custom",
                    backend=LightSimBackend(),
                    reward_class=GameplayReward,
                    )
obs = env.reset()
obs, reward, done, info = env.step(env.action_space())
reward, info

(1.0,
 {'disc_lines': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1], dtype=int32),
  'is_illegal': False,
  'is_ambiguous': False,
  'is_dispatching_illegal': False,
  'is_illegal_reco': False,
  'reason_alarm_illegal': None,
  'opponent_attack_line': None,
  'opponent_attack_sub': None,
  'opponent_attack_duration': 0,
  'exception': [],
  'rewards': {}})

In [126]:
from l2rpn_baselines.PPO_SB3.utils import SB3Agent
from grid2op.gym_compat import BoxGymActSpace, BoxGymObsSpace
from GymEnvWithRecoWithDNWithShuffle import GymEnvWithRecoWithDNWithShuffle
from run_trained_agents import BaselineAgent
from stable_baselines3 import PPO

def get_agent(weights_dir, env, safe_max_rho=0.2, limit_cs_margin=-1):
    """this is basically a copy paste of the PPO_SB3 evaluate function with some minor modification
    used to load the correct weights
    """
    agent_dir = os.path.dirname(weights_dir)
    submission_dir = os.path.dirname(agent_dir)

    # compute the score of said agent
    with open(os.path.join(submission_dir, "preprocess_obs.json"), 'r', encoding="utf-8") as f:
        obs_space_kwargs = json.load(f)
    obs_space_kwargs["functs"] = {"storage_capacity_setpoint":
                                        (lambda grid2op_obs: np.full(env.n_storage, env.parameters.INIT_STORAGE_CAPACITY), 0., 1.0, None, None)}
    with open(os.path.join(submission_dir, "preprocess_act.json"), 'r', encoding="utf-8") as f:
        act_space_kwargs = json.load(f)
    
    # load the attributes kept
    with open(os.path.join(agent_dir, "obs_attr_to_keep.json"), encoding="utf-8", mode="r") as f:
        obs_attr_to_keep = json.load(fp=f)
    with open(os.path.join(agent_dir, "act_attr_to_keep.json"), encoding="utf-8", mode="r") as f:
        act_attr_to_keep = json.load(fp=f)

    # create the action and observation space
    gym_observation_space =  BoxGymObsSpace(env.observation_space,
                                            attr_to_keep=obs_attr_to_keep,
                                            **obs_space_kwargs)
    gym_action_space = BoxGymActSpace(env.action_space,
                                      attr_to_keep=act_attr_to_keep,
                                      **act_space_kwargs)

    # apply normalization on created gym_observation_space and gym_action_space
    for attr_nm in act_attr_to_keep:
            if (("multiply" in act_space_kwargs and attr_nm in act_space_kwargs["multiply"]) or 
                ("add" in act_space_kwargs and attr_nm in act_space_kwargs["add"]) 
               ):
                # attribute is scaled elsewhere
                continue
            gym_action_space.normalize_attr(attr_nm)
    
    for attr_nm in obs_attr_to_keep:
            if (("divide" in obs_space_kwargs and attr_nm in obs_space_kwargs["divide"]) or 
                ("subtract" in obs_space_kwargs and attr_nm in obs_space_kwargs["subtract"]) 
               ):
                # attribute is scaled elsewhere
                continue
            gym_observation_space.normalize_attr(attr_nm)
    
    # create the gym environment for the PPO agent...
    gymenv = GymEnvWithSetPoint(env, safe_max_rho=float(safe_max_rho), shuffle_chronics=False)    
    gymenv.action_space.close()
    gymenv.action_space = gym_action_space
    gymenv.observation_space.close()
    gymenv.observation_space = gym_observation_space
    
    # create a grid2gop agent based on that (this will reload the save weights)
    l2rpn_agent = SB3Agent(env.action_space,
                           gym_action_space,
                           gym_observation_space,
                           nn_path=weights_dir,
                           gymenv=gymenv
                           )
    
    agent_to_evaluate = BaselineAgent(l2rpn_agent, limit_cs_margin)
    return agent_to_evaluate

In [127]:
# path_agents = "./saved_model/expe_case_14/expe_hp/"
path_agents = "./saved_model/expe_case_14/expe_to_run/"
machine_dir = "case14_random_stor_setpoint"

root_dir = os.path.abspath(path_agents)
submission_dir  = os.path.join(root_dir, machine_dir)
list_names = [name for name in os.listdir(submission_dir) if name.endswith("0")]
name = list_names[0]
print("name:", name)

training_iter = 5_000_000

agent_dir = os.path.join(submission_dir, name)
weights_dir = os.path.join(agent_dir, f"{name}_{training_iter}_steps.zip")

agent = get_agent(weights_dir,
                                env,
                                0.2,
                                -1)
gym_env = agent.l2rpn_agent.gymenv
agent

name: case14_random_stor_setpoint_20230228_151131_gymenv_kwargs_0
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [183]:
NB_EPISODE = len(env.chronics_handler.real_data.available_chronics())

def run_episodes(agent, nb_episode=NB_EPISODE):
    gymenv = agent.l2rpn_agent.gymenv
    dict_res = {}
    for ep_id in range(nb_episode):
        chronic_name = gym_env.init_env.chronics_handler.get_name()
        dict_res[chronic_name] = {}
        all_obs, all_gymobs, all_rews, all_rews_bu, all_acts, all_ts = [], [], [], [], [], []
        for key in ("all_obs", "all_gymobs", "all_rews", "all_rews_bu", "all_acts", "all_ts"):
            dict_res[chronic_name][key] = locals()[key]
        

        # Initialisation
        gymenv.reset()
        dict_res[chronic_name]["all_storage_setpoints"] = gymenv.storage_setpoint
        obs = gymenv.init_env.get_obs()
        reward = env.reward_range[0]
        done = False
        # Save
        all_obs.append(obs)
        all_gymobs.append(gymenv.get_obs())
        all_ts.append(gymenv.init_env.nb_time_step)
        all_rews.append(reward)
        all_rews_bu.append(reward)
        with tqdm(total=env.chronics_handler.max_timestep(), disable=False) as pbar:
            while True:
                # Apply the action of the agent
                action = agent.act(obs, reward, done)   
                obs, reward, done, info = gymenv.init_env.step(action)
                # Update reward
                all_rews_bu.append(reward)
                reward = gymenv._update_reward(obs, reward)
                # Apply heuristic
                if not done:
                    obs, reward, done, info = gymenv.apply_heuristics_actions(obs, reward, done, info)
                # Update gym obs
                gymobs_tmp = gymenv.observation_space.to_gym(obs)
                gymobs = gymenv._update_setpoint(gymobs_tmp, "storage_capacity_setpoint", gymenv.storage_setpoint[gymenv.init_env.nb_time_step, :])
                gymenv._last_obs = gymobs
                # Save
                all_obs.append(obs)
                all_gymobs.append(gymenv.get_obs())
                all_acts.append(action.set_storage)
                all_rews.append(reward)
                all_ts.append(gymenv.init_env.nb_time_step)
                pbar.update(1)
                if done:
                    break
        all_acts.append(np.full(env.n_storage, np.nan))
    return dict_res

/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [184]:
dict_res = run_episodes(agent)
dict_res.keys()

100%|██████████| 288/288 [00:00<00:00, 322.23it/s]


dict_keys(['2019-01-13', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-12'])

In [185]:
chronic_name = "2019-01-12"
dict_chronic = dict_res["2019-01-12"]
dict_chronic.keys()

/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['all_obs', 'all_gymobs', 'all_rews', 'all_rews_bu', 'all_acts', 'all_ts', 'all_storage_setpoints'])

In [187]:
pd.DataFrame(dict_chronic["all_acts"], columns = ["set_storage_0", "set_storage_1"]) # nan for action at final ts
pd.DataFrame({col:dict_chronic[col] for col in ("all_ts", "all_rews", "all_rews_bu")})
pd.DataFrame(dict_chronic["all_storage_setpoints"], columns = ["storage_setpoint_0", "storage_setpoint_1"])

/home/boguslawskieva/miniconda3/envs/baselines_dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,set_storage_0,set_storage_1
0,-0.417069,0.618999
1,-0.320884,1.385710
2,-0.313392,1.183510
3,-0.413376,1.695843
4,-0.430983,1.272136
...,...,...
284,-0.101097,-0.985636
285,-0.220069,0.238733
286,-0.173903,-0.184022
287,-0.793038,-0.529568


In [134]:
dict_chronic["all_gymobs"][0], dict_chronic["all_storage_setpoints"][0][0,:], (dict_chronic["all_obs"][0].storage_charge - env.storage_Emin)/(env.storage_Emax - env.storage_Emin), dict_chronic["all_obs"][0].storage_power

(array([ 0.43522218,  0.43620348, -0.7250852 , -0.62380534, -0.6252446 ,
         0.6587491 ,  0.01473814,  0.03032282, -0.03185767,  0.0211238 ,
         0.08461636,  0.063299  , -0.01329887, -0.05826628, -0.01852429,
        -0.05348825,  0.06889462, -0.0191873 , -0.03178546, -0.07468337,
        -0.05994064,  0.06419131,  0.05869201,  0.19158846, -0.00789621,
         0.00799766,  0.45225596,  0.5357014 ,  0.5       ,  0.5       ,
         0.5       ,  0.5       ], dtype=float32),
 array([0.45225597, 0.53570137]),
 array([0.5, 0.5], dtype=float32),
 array([0., 0.], dtype=float32))

In [150]:
attr_index = np.where(np.array(gym_env.observation_space._attr_to_keep) == "storage_capacity_setpoint")[0][0]
dims = [0] + gym_env.observation_space._dims
dict_chronic["all_gymobs"][0][dims[attr_index]:dims[attr_index+1]]

array([0.45225596, 0.5357014 ], dtype=float32)